In [1]:
import pandas as pd

In [2]:
new_scholar_works = pd.read_csv('new_scholar_works.csv')

In [4]:
new_scholar_works.sample(25)

,authors,collaborators,titles,dates,abstracts
2273,"Croft, Anna K.","Dowling, Daniel P.; Vey, Jessica L.; Croft, An...",Structural Diversity In The Adomet Radical Enz...,2012,AdoMet radical enzymes are involved in process...
11857,"Young, Neal E.","Chrobak, Marek; Kenyon, Claire; Noga, John; Yo...",Incremental Medians Via Online Bidding,2007,In the k-median problem we are given sets of f...
15,"Abrego, Bernardo","Aichholzer, Oswin; Abrego, Bernardo; Fernández...",There is a unique crossing-minimal rectilinear...,2011,"We show that, up to isomorphism, there is a un..."
9993,"Stauffer, John","Stauffer, John; Cody, Ann Marie; McGinnis, Pau...",CSI 2264: characterizing young stars in NGC 22...,2015,We identify nine young stellar objects (YSOs) ...
8159,"Patel, Bharat","Huang, Chi-Yu; Patel, Bharat; Mah, Robert; Bar...","Caldicellulosiruptor owensensis sp. nov., an a...",1998,"An anaerobic, extremely thermophilic, xylanoly..."
11589,"Wu, Fang-Xiang","Hang, Xiyi; Wu, Fang-Xiang",Sparse Representation for Classification of Tu...,2009,Personalized drug design requires the classifi...
3404,"Freedman, Robert I.","Childress, William J.; Freedman, Robert I.; Ko...",Surface Characteristics of Separated Subpopula...,1979,Specific subpopulations of 129/J mouse teratoc...
1185,"Brgoch, Jakoah","Miao, Mao-Sheng; Wang, Xiao-Li; Brgoch, Jakoah...",Anionic Chemistry Of Noble Gases: Formation Of...,2015,"While often considered to be chemically inert,..."
2767,"Dubremetz, Jean-Francois","Bermudes, David; Dubremetz, Jean-Francois; Ach...",Cloning of a cDNA encoding the dense granule p...,1994,GRA3 is a 30-kDa protein located inside the de...
3559,"Garcia, Sandra Gloria","Rothstein-Fisch, Carrie; Trumbull, Elise; Garc...",Making the implicit explicit: Supporting teach...,2009,"In this paper, we report the results of a long..."


In [5]:
authors = new_scholar_works['authors'].tolist()

In [8]:
authors
full_names = []
for full_name in authors:
    if ',' not in full_name:
        print(full_name)

A.C. Asperti
Abdeljawad
Alex Venn
Alexander A. Aarts
Alexander B. Swan
Alexandra Gerbasi
Amanda L. Forest
Amar Nancy
Amos Anastasia
Andersen St
Andreas Cordes
Andreas Gloeckner
Andrew M. Crain
Angela Attwood
Angela Dorrough
Ann Calhoun-Sauls
Anna Dreber
Anna Fedor
Anna Karapiperi
Annick Bosch
Antunez MY
Anup Gampa
Author Vitae
Barney L Bales
Barney L Bales
Belen Fernandez-Castilla
Bellovarym J.
Beloborodov
Benjamin T. Brown
Bobby Den Bezemer
Brown Darren
Bryan Gorges
C. E. Spivak
Carolina Vautier-Giongo
Carolina Vautier-Giongo
Casey Eggleston
Catherine Classen
Catherine Classen
Cheryl Koopman
Cheryl Koopman
Christina J. Campbell
Christopher J. Anderson
Christopher R. Chartier
Chun Helen H.
Cody D. Christopherson
D.A. Gray
D.B. Weismann
David E. Knapp
David G. Dobolyi
Denny Borsboom
Dick Van Oevelen
Dimitris Assimakopoulos
Djellouli
Douglas Gazarian
Dwivedi
Ed Cremata
Elisa Maria Galliani
Elizabeth Bartmess
Elizabeth Chagnon
Elizabeth Gilbert
Elizabeth M. Bollen
Elizabeth Mcgarvey
Eliza